In [ ]:
import pandas as pd
from pyarrow.parquet import ParquetFile
import dask.dataframe as dd
import os
import xarray as xr

#%matplotlib inline


In [ ]:
dir_files = [os.path.join("..\short_range_18files", files) for files in os.listdir("..\short_range_18files")]
#dir_files = [os.path.join("short_range_2files", files) for files in os.listdir("short_range_2files")]
print(dir_files)

In [ ]:
fds = []
for f in dir_files:
    fds.append(xr.open_dataset(f))

In [ ]:
%%time
ds = xr.concat(fds,dim="time")

In [ ]:
df = ds.to_dataframe()
df = df.streamflow
df

In [ ]:
%%time
df = pd.Series.to_frame(df)

df.to_parquet("../data/parquet_all_feature_ids_pyarrow_snappy.parquet"
             , engine="pyarrow" #"pyarrow", "auto"
             , compression="snappy" # "gzip"
            )


In [ ]:
ParquetFile("../data/parquet_all_feature_ids_pyarrow_snappy.parquet").metadata #  num_columns: 3

In [ ]:
%%time
data = dd.read_parquet(
    "../data/parquet_all_feature_ids_pyarrow_snappy.parquet"
    , storage_options={"anon": True}
    )
data
result = data.compute()
#result = result.loc[:,:, [101]]
#result= result.loc[:, :, 100:1032]
result= result.loc[:, :, 100:252765]
#result
r_xa = result.to_xarray()
r_xa
r_xa.plot.scatter("time","streamflow")